# I - Importation

## 1. Imporation des modules standards

In [1]:
import importlib
import os
import sys

import datetime          as dt
import numpy             as np
import pandas            as pd

import seaborn           as sns
import matplotlib.pyplot as plt

from math import ceil

## 2. Importation d'un module personnalisé

In [11]:
## Importation d'un module personnalisé
sys.path.insert(0, r'D:\Users\User\Documents\Works\03 - Formations\02 - DataScientest\06 - Project\01 - Modules')

import BetPy_Annex
importlib.reload(BetPy_Annex)

from BetPy_Annex import Collect_results, Collect_infos, Aggregate_infos, Exploit_infos

## 3. Imporations des données + prétraitement

In [16]:
## Chargement des données
os.chdir(r'D:\Users\User\Documents\Works\03 - Formations\02 - DataScientest\06 - Project\00 - Data\Football')

club        = pd.read_csv('club.csv', index_col='id')
match       = pd.read_csv('match.csv', index_col='id')
match_event = pd.read_csv('match_event.csv').sort_values(by=['match_id', 'matchperiod', 'eventsec'])
player      = pd.read_csv('player.csv', index_col='id')
referee     = pd.read_csv('referee.csv', index_col='id')

In [17]:
## Correction des noms de certains clubs
dico_correct = dict()
dico_correct['Angers SCO']                = 'Angers'
dico_correct['Atl\\u00e9tico Madrid']     = 'Atlético Madrid'
dico_correct['Bayern M\\u00fcnchen']      = 'Bayern München'
dico_correct['Deportivo Alav\\u00e9s']    = 'Deportivo Alavés'
dico_correct['Deportivo La Coru\\u00f1a'] = 'Deportivo La Coruña'
dico_correct['K\\u00f6ln']                = 'Köln'
dico_correct['Legan\\u00e9s']             = 'Leganés'
dico_correct['Saint']                     = 'Saint-étienne'
dico_correct['Saint-\\u00c9tienne']       = 'Saint-étienne'
dico_correct['\\u00c9tienne']             = 'Saint-étienne'

club.name                                   = club.name.replace(dico_correct)
match[['home_club', 'away_club', 'winner']] = match[['home_club', 'away_club', 'winner']].replace(dico_correct)

# Correction des lignes concernant les matchs à domicile de Saint-étienne
dico_correct = dict()
dico_correct['2017-08-05 18:00:00'] = 'Nice'
dico_correct['2017-08-19 18:00:00'] = 'Amiens SC'
dico_correct['2017-10-14 18:00:00'] = 'Metz'
dico_correct['2017-10-20 18:45:00'] = 'Montpellier'
dico_correct['2017-11-05 20:00:00'] = 'Olympique Lyonnais'
dico_correct['2017-12-15 19:45:00'] = 'Monaco'
dico_correct['2018-01-14 14:00:00'] = 'Toulouse'
dico_correct['2018-01-27 19:00:00'] = 'Caen'
dico_correct['2018-03-18 16:00:00'] = 'Guingamp'
dico_correct['2018-04-22 15:00:00'] = 'Troyes'
dico_correct['2018-05-06 13:00:00'] = 'Bordeaux'
dico_correct['2018-05-19 19:00:00'] = 'Lille'

cond = (match.home_club == 'Saint-étienne') & (match.dateutc.isin(dico_correct.keys()))
match.loc[cond, 'away_club'] = match.loc[cond, 'dateutc'].apply(lambda x: dico_correct[x])

## Re-typage + tri
match.dateutc = match.dateutc.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
match = match.sort_values(by=['dateutc'])

## Récupération du poste de chaque joueur dans la table « match_event » depuis la table « player »
match_event = match_event.join(player['position'], on='players_id')

In [12]:
## Chargement de la table « match_results » si celle-ci existe déjà
try:
    match_results = pd.read_csv('match_results.csv', index_col=0)
except FileNotFoundError:
    print("/!\ Le fichier « match_results.csv » n'existe pas encore !")

## Chargement de la table « match_infos » si celle-ci existe déjà
try:
    match_infos = pd.read_csv('match_infos.csv', index_col=0)
except FileNotFoundError:
    print("/!\ Le fichier « match_infos.csv » n'existe pas encore !")

## Chargement de la table « match_all » si celle-ci existe déjà
try:
    match_all = pd.read_csv('match_all.csv', index_col=0)
except FileNotFoundError:
    print("/!\ Le fichier « match_all.csv » n'existe pas encore !")

## Chargement de la table « data » si celle-ci existe déjà
try:
    data = pd.read_csv('data.csv', index_col=0)
except FileNotFoundError:
    print("/!\ Le fichier « data.csv » n'existe pas encore !")

# II - Création des tables contenant les variables explicatives

## 1. Création de la table « match_results » (extraction depuis la table « match »)

> <strong>Description :</strong>
>> ...

In [18]:
## Création de la table « match_results »
match_results = Collect_results(match)

In [19]:
## Affichage pour vérification
match_results[-6:].T

,2674,2675,2676,2677,2678,2679
Date,2018-05-20 18:45:00,2018-05-20 18:45:00,2018-05-20 18:45:00,2018-05-20 18:45:00,2018-05-20 18:45:00,2018-05-20 18:45:00
Club,Barcelona,Real Sociedad,Sassuolo,Roma,Lazio,Internazionale
Club_ADV,Real Sociedad,Barcelona,Roma,Sassuolo,Internazionale,Lazio
Période_jour,4,4,4,4,4,4
Arbitre_id,383,383,338,338,356,356
Match_id,760,760,1,1,0,0
Dom_Ext,1,0,1,0,1,0
CLB_Série_vic,26,14,11,22,21,19
CLB_Tot_buts_pour,81,58,21,55,77,57
CLB_Tot_buts_contre,17,50,50,23,36,22


In [28]:
# Écriture de la table « match_results »
match_results.to_csv('match_results.csv', sep=',')

## 2. Création de la table « match_infos » (extraction depuis la table « match_event »)

> <strong>Description :</strong>
>> ...

In [49]:
## Vérification des informations produites par la procédure « Collect_infos »

## Sélection du match pour vérification
match_id = 985

## Lancement de la procédure « Collect_infos » + récupération de la sortie
current_match = match_event[match_event.match_id == match_id].sort_values(by=['matchperiod', 'eventsec'])
meta_data, infos = Collect_infos(current_match)

## Sélection des joueurs pour vérification
players = [str(p) for p in meta_data.keys()]
players = sorted(players[7:])

## Traitement de la sortie pour un meilleur affichage
types      = ['Activité_1H', 'Arrêts_1H', 'Fautes_1H', 'Passes_1H', 'Tirs_1H',
              'Activité_2H', 'Arrêts_2H', 'Fautes_2H', 'Passes_2H', 'Tirs_2H']
to_display = []
for player in players:
    to_display += infos.filter(regex='.*_' + player + '_.*', axis=1).values.tolist()

## Affichage
pd.DataFrame(to_display, index=players).set_axis(types, axis=1).T

,1460,150,161,1673,1739,1839,2065,2162,2215,282,287,304,3217,3234,3235,359,378,442,575,71
Activité_1H,26.0,14.0,15.0,18.0,16.0,12.0,35.0,50.0,0.0,28.0,30.0,0.0,0.0,32.0,0.0,31.0,20.0,26.0,0.0,24.0
Arrêts_1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fautes_1H,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Passes_1H,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tirs_1H,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Activité_2H,24.0,11.0,21.0,37.0,23.0,9.0,28.0,34.0,20.0,44.0,25.0,12.0,29.0,45.0,5.0,45.0,19.0,28.0,4.0,34.0
Arrêts_2H,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fautes_2H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Passes_2H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tirs_2H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
match_infos = None

## Boucle sur l'ensemble des matchs présent dans la table « match_results »
for match_id in match_results.Match_id.unique():
    # Récupération des événements du match courant
    current_match = match_event[match_event.match_id == match_id].sort_values(by=['matchperiod', 'eventsec'])
    
    # Collecte des informations sur le match courant
    meta_infos, infos = Collect_infos(current_match)
    
    # Agrégation des informations collectées ci-avant
    if match_infos is None:
        n_index, match_infos = Aggregate_infos(match_id, meta_infos, infos)
    else:
        n_index, stats = Aggregate_infos(match_id, meta_infos, infos, n_index)
        match_infos    = match_infos.append(stats)

## Ré-ordination des colonnes
match_infos = match_infos.reindex(sorted(match_infos.columns, reverse=True), axis=1)

In [6]:
## Vérification des informations produites par la procédure « Aggregate_infos »

## Sélection du match à regarder
match_id = 985

## Affichage
match_infos[match_infos.Match_id == match_id].T

,1228,1229
Équipe_id,59.0,6.0
Match_id,985.0,985.0
(Tirs_Équipe_2H),1.0,0.0
(Tirs_Équipe_1H),2.0,0.0
(Tirs_Milieux_2H),0.0,0.0
(Tirs_Milieux_1H),1.0,0.0
(Tirs_Défenseurs_2H),0.0,0.0
(Tirs_Défenseurs_1H),0.0,0.0
(Tirs_Attaquants_2H),1.0,0.0
(Tirs_Attaquants_1H),1.0,0.0


In [10]:
# Écriture de la table « match_infos »
match_infos.to_csv('match_infos.csv', sep=',')

## 3. Augmentation de la table « match_infos »

> <strong>Description :</strong>
>> ...

In [13]:
## Récupération des noms des clubs dans la table « match_infos » depuis la table « club »
match_infos.insert(2, 'Club', match_infos.join(club.name, on='Équipe_id').name)

## Récupération de la date des match dans la table « match_infos » depuis la table « match_results »
match_infos.insert(0, 'Date', pd.merge(match_results[['Match_id', 'Club', 'Date']], match_infos[['Match_id', 'Club']], on=['Match_id', 'Club']).Date)

## Tri sur le champs 'Date'
match_infos = match_infos.sort_values('Date').reset_index(drop=True)

In [14]:
## Traitement des informations présentes dans « match_infos » + création de variables explicatives
match_infos = pd.concat([match_infos, Exploit_infos(match_infos)], axis=1)

In [9]:
## Affichage
match_infos[match_infos.Club == 'Toulouse'].T

,0,32,117,156,278,463,565,636,810,953,...,1632,1693,2011,2158,2197,2351,2432,2517,2597,2645
Date,2017-08-04 18:45:00,2017-08-12 18:00:00,2017-08-25 18:15:00,2017-08-27 12:30:00,2017-09-16 14:15:00,2017-10-01 10:30:00,2017-10-21 13:30:00,2017-10-25 18:45:00,2017-11-18 16:00:00,2017-12-02 12:30:00,...,2018-02-17 19:00:00,2018-02-24 19:45:00,2018-04-07 14:00:00,2018-04-21 13:30:00,2018-04-22 18:45:00,2018-05-12 13:15:00,2018-05-19 16:30:00,NaN,NaN,NaN
Équipe_id,86,86,86,86,86,86,86,86,86,86,...,86,86,86,86,86,86,86,86,86,86
Match_id,759,746,730,727,709,690,678,665,647,624,...,526,519,470,451,447,427,412,406,392,388
Club,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,...,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse,Toulouse
(Tirs_Équipe_2H),0,2,0,5,4,0,6,0,0,0,...,0,0,7,0,9,4,0,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tirs_Tot_2H_3,NaN,NaN,NaN,2,7,9,9,10,6,6,...,8,8,8,7,7,16,13,13,5,1
Tirs_Tot_2H_1,NaN,0,2,0,5,4,0,6,0,0,...,8,0,0,7,0,9,4,0,1,0
Tirs_Tot_3H_5,NaN,NaN,NaN,NaN,NaN,23,23,28,28,21,...,28,28,18,27,27,28,40,37,33,33
Tirs_Tot_3H_3,NaN,NaN,NaN,7,14,16,16,21,12,12,...,15,15,18,12,12,25,28,28,17,5


In [20]:
# Écriture de la table « match_infos » augmentée
match_infos.to_csv('match_infos_aug.csv', sep=',')

## 4. Création de la table « match_all »

> <strong>Description :</strong>
>> ...

In [29]:
## Jointure entre les tables « match_results » et « match_infos »
match_all = pd.merge(match_results, match_infos.drop(['Date', 'Équipe_id'], axis=1), on=['Match_id', 'Club'])

## Auto-jointure pour la récupération des variables explicatives concernant les clubs adverses
match_all = pd.merge(match_all, match_all.filter(regex='(Date|Club$|^Act_.*|^Gar_.*|^Fte_.*|^Pass_.*|^Tirs_.*)', axis=1).rename(columns={'Club': 'Club_ADV'}), how='inner', on=['Date', 'Club_ADV'])

## Fonction pour le renommage des colonnes après l'auto-jointure
def renameCols(name):
    if name[-2:] == '_x':
        return 'CLB_' + name[:-2]
    elif name[-2:] == '_y':
        return 'ADV_' + name[:-2]
    else:
        return name

## Renommage des colonnes
match_all.columns = match_all.columns.map(renameCols)

In [32]:
## Affichage
match_all.iloc[-2:, :].T

,2284,2285
Date,2018-05-20 18:45:00,2018-05-20 18:45:00
Club,Lazio,Internazionale
Club_ADV,Internazionale,Lazio
Période_jour,4,4
Arbitre_id,356,356
...,...,...
ADV_Tirs_Tot_2H_3,16,5
ADV_Tirs_Tot_2H_1,11,0
ADV_Tirs_Tot_3H_5,42,33
ADV_Tirs_Tot_3H_3,23,12


In [31]:
# Écriture de la table « match_all »
match_all.to_csv('match_all.csv', sep=',')

In [33]:
## TEST
match_all.iloc[-2:, :].T.to_csv('match_infos__TEST2.csv', sep=',')

## 5. Création de la table « data »

> <strong>Description :</strong>
>> ...

In [14]:
## Suppression des lignes ayant des NA dans la table « match_all »
match_all = match_all.dropna()

## Création de la table « data »
data                          = pd.DataFrame(match_all['Dom_Ext'])

data['Écart_buts_diff']       = match_all['CLB_Tot_buts_diff']          - match_all['ADV_Tot_buts_diff']
data['Écart_buts_pour']       = match_all['CLB_Tot_buts_pour']          - match_all['ADV_Tot_buts_pour']
data['Écart_buts_contre']     = match_all['ADV_Tot_buts_contre']        - match_all['CLB_Tot_buts_contre']

data['Écart_Poss_moy_éq_1H']  = match_all['CLB_Poss_moy_équipe_1H']     - match_all['ADV_Poss_moy_équipe_1H']
data['Écart_Poss_moy_att_1H'] = match_all['CLB_Poss_moy_attaquants_1H'] - match_all['ADV_Poss_moy_attaquants_1H']
data['Écart_Poss_moy_mil_1H'] = match_all['CLB_Poss_moy_milieux_1H']    - match_all['ADV_Poss_moy_milieux_1H']
data['Écart_Poss_moy_def_1H'] = match_all['CLB_Poss_moy_défenseurs_1H'] - match_all['ADV_Poss_moy_défenseurs_1H']
data['Écart_Poss_moy_gar_1H'] = match_all['CLB_Poss_moy_gardiens_1H']   - match_all['ADV_Poss_moy_gardiens_1H']
data['Écart_Poss_moy_éq_2H']  = match_all['CLB_Poss_moy_équipe_2H']     - match_all['ADV_Poss_moy_équipe_2H']
data['Écart_Poss_moy_att_2H'] = match_all['CLB_Poss_moy_attaquants_2H'] - match_all['ADV_Poss_moy_attaquants_2H']
data['Écart_Poss_moy_mil_2H'] = match_all['CLB_Poss_moy_milieux_2H']    - match_all['ADV_Poss_moy_milieux_2H']
data['Écart_Poss_moy_def_2H'] = match_all['CLB_Poss_moy_défenseurs_2H'] - match_all['ADV_Poss_moy_défenseurs_2H']
data['Écart_Poss_moy_gar_2H'] = match_all['CLB_Poss_moy_gardiens_2H']   - match_all['ADV_Poss_moy_gardiens_2H']
data['Écart_Poss_moy_éq_3H']  = match_all['CLB_Poss_moy_équipe_3H']     - match_all['ADV_Poss_moy_équipe_3H']
data['Écart_Poss_moy_att_3H'] = match_all['CLB_Poss_moy_attaquants_3H'] - match_all['ADV_Poss_moy_attaquants_3H']
data['Écart_Poss_moy_mil_3H'] = match_all['CLB_Poss_moy_milieux_3H']    - match_all['ADV_Poss_moy_milieux_3H']
data['Écart_Poss_moy_def_3H'] = match_all['CLB_Poss_moy_défenseurs_3H'] - match_all['ADV_Poss_moy_défenseurs_3H']
data['Écart_Poss_moy_gar_3H'] = match_all['CLB_Poss_moy_gardiens_3H']   - match_all['ADV_Poss_moy_gardiens_3H']

data['[Résultat]']            = match_all['[Résultat]']

## Affichage
data

,Dom_Ext,Écart_buts_diff,Écart_buts_pour,Écart_buts_contre,Écart_Poss_moy_éq_1H,Écart_Poss_moy_att_1H,Écart_Poss_moy_mil_1H,Écart_Poss_moy_def_1H,Écart_Poss_moy_gar_1H,Écart_Poss_moy_éq_2H,Écart_Poss_moy_att_2H,Écart_Poss_moy_mil_2H,Écart_Poss_moy_def_2H,Écart_Poss_moy_gar_2H,Écart_Poss_moy_éq_3H,Écart_Poss_moy_att_3H,Écart_Poss_moy_mil_3H,Écart_Poss_moy_def_3H,Écart_Poss_moy_gar_3H,[Résultat]
426,1.0,6.0,1.0,5.0,1.866,10.354,2.644,-9.668,-3.324,1.980,10.156,-2.310,-11.810,3.970,1.923,10.255,0.167,-10.739,0.323,1.0
427,0.0,-6.0,-1.0,-5.0,-1.866,-10.354,-2.644,9.668,3.324,-1.980,-10.156,2.310,11.810,-3.970,-1.923,-10.255,-0.167,10.739,-0.323,0.0
448,1.0,-3.0,-2.0,-1.0,-7.784,-6.510,-2.278,14.446,-5.652,-6.282,-3.476,-19.334,11.020,11.790,-7.033,-4.993,-10.806,12.733,3.069,1.0
449,0.0,3.0,2.0,1.0,7.784,6.510,2.278,-14.446,5.652,6.282,3.476,19.334,-11.020,-11.790,7.033,4.993,10.806,-12.733,-3.069,0.0
462,1.0,3.0,0.0,3.0,8.648,1.032,2.760,4.348,-8.140,6.170,0.252,11.912,4.616,-16.776,7.409,0.642,7.336,4.482,-12.458,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,0.0,-56.0,-23.0,-33.0,-5.908,3.808,-2.742,-8.926,7.860,-2.708,2.496,-6.412,-5.138,9.054,-4.308,3.152,-4.577,-7.032,8.457,0.0
2676,1.0,-61.0,-34.0,-27.0,1.036,1.172,-21.674,15.116,5.388,7.220,3.190,-16.428,17.644,-4.412,4.128,2.181,-19.051,16.380,0.488,0.0
2677,0.0,61.0,34.0,27.0,-1.036,-1.172,21.674,-15.116,-5.388,-7.220,-3.190,16.428,-17.644,4.412,-4.128,-2.181,19.051,-16.380,-0.488,1.0
2678,1.0,6.0,20.0,-14.0,1.780,27.564,-22.840,-0.646,-4.078,-2.936,23.670,-17.050,-4.918,-1.700,-0.578,25.617,-19.945,-2.782,-2.889,0.0


In [15]:
# Écriture de la table « data »
data.to_csv('data.csv', sep=',')

# III - Séparation du Jeux De Données

In [2]:
## Suppression des lignes pour lesquels il manque des données
data = data.dropna()

## Séparation des données pour entraînement des modèles
#  -> les données sont triées dans l'ordre où les matchs se sont déroulés

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=.2, shuffle=False)

## /!\ Le mot-clef « shuffle » de la fonction « train_test_split » permet de
##     faire la séparation sans mélanger les données. Ce qui permet de se passer
##     des morceaux de codes ci-dessous.
# X_train = data.drop('[Résultat]', axis=1).iloc[:int(ceil(.8 * len(data))),  :]
# X_test  = data.drop('[Résultat]', axis=1).iloc[int(ceil(-.2 * len(data))):, :]
# y_train = data[['[Résultat]']].iloc[:int(ceil(.8 * len(data))),  :]
# y_test  = data[['[Résultat]']].iloc[int(ceil(-.2 * len(data))):, :]

NameError: name 'data' is not defined

# IV - Entraînement des modèles

## 1. Importaiton des modules standards

In [1]:
from sklearn import preprocessing, model_selection, svm, neighbors, ensemble